In [1]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import pandas as pd
#import matlplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import  RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_predict
from sklearn.utils import class_weight

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import TensorBoard, CSVLogger,  ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

2.19.0


In [18]:
# Paramètres
#Escherichia coli
#Staphylococcus
#Bacillus
#Lactobacillus
#Salmonella
#nb de folds+ 5 espèces dans train/test+ 9 souches à controler dans les data_means

n_species = 5        # nombre d'espèces
n_strains_per_species = 4  # nombre de souches par espèce ex E-coli 15 .  K-12, O157:H7, B, C
n_replicates = 9     # réplicats par souche
n_features = 50      # nombre de colonnes/features (spectres)

#chaque feature est une absorbance à une longueur d’onde spécifique.
# Feature_0 = absorbance à 400 nm, Feature_1 = absorbance à 410 nm...

np.random.seed(42)

# Création des données
data_list = []
labels = []
strain_ids = []

for species_id in range(n_species):
    for strain_id in range(n_strains_per_species):
        # Valeurs de base pour la souche
        base = np.random.rand(n_features) * 10
        for rep in range(n_replicates):
            # Ajouter un peu de bruit pour chaque réplicat
            data_list.append(base + np.random.normal(0, 1, n_features))
            labels.append(f"Species_{species_id}")
            strain_ids.append(f"Strain_{species_id}_{strain_id}")

# Convertir en DataFrame
data = pd.DataFrame(data_list, columns=[f"Feature_{i}" for i in range(n_features)])
data["species"] = labels
data["strain"] = strain_ids


label_df= data[['strain', 'species']]
data=data.iloc[:,0:50]
print("Données :")
print(data.head())
print("label")
print(label_df.head())

Données :
   Feature_0  Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  \
0   4.483868   9.678511   7.204291   5.685481   0.081664   0.840101   
1   3.842479  10.475788   6.617886   5.658923   1.168078   0.096430   
2   3.972861  10.814286   5.712456   6.171219   1.820069   2.341768   
3   3.298886  10.363542   7.534033   4.740846   1.733367   1.945263   
4   2.952880   9.392407   7.824927   6.852340   0.359890   1.225444   

   Feature_6  Feature_7  Feature_8  Feature_9  ...  Feature_40  Feature_41  \
0   0.120197   9.718884   6.354768   5.317686  ...    1.000710    5.308882   
1   0.876956   8.922817   6.016264   6.846139  ...   -0.330281    5.020332   
2  -0.656115   7.341305   6.533092   7.377710  ...    1.846050    4.094612   
3  -0.303021   8.815487   6.069359   5.937755  ...    0.490016    5.168228   
4   0.105891   8.008432   7.776604   7.485707  ...    1.333900    5.613900   

   Feature_42  Feature_43  Feature_44  Feature_45  Feature_46  Feature_47  \
0    1.821779    

In [19]:

key = label_df['species'].value_counts().keys().tolist()
print(pd.unique(label_df['strain'] ))

['Strain_0_0' 'Strain_0_1' 'Strain_0_2' 'Strain_0_3' 'Strain_1_0'
 'Strain_1_1' 'Strain_1_2' 'Strain_1_3' 'Strain_2_0' 'Strain_2_1'
 'Strain_2_2' 'Strain_2_3' 'Strain_3_0' 'Strain_3_1' 'Strain_3_2'
 'Strain_3_3' 'Strain_4_0' 'Strain_4_1' 'Strain_4_2' 'Strain_4_3']


In [32]:
from sklearn.utils import class_weight

#chargement des données
#data = np.genfromtxt('/Users/anath/Master_SSD/Semestre3/Fouille_donnees/dataset_challenge/spectra-train.csv', delimiter = ";")
#label_df = pd.read_csv('/Users/anath/Master_SSD/Semestre3/Fouille_donnees/dataset_challenge/meta-train.csv', delimiter = ";")

key = label_df['species'].value_counts().keys().tolist() #species names to list
valeur = label_df['species'].value_counts().tolist()
##encode y
y = label_df['species']
#encode y
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

#pour prendre en compte le déséquilibre des classes
#pour calculer les weights
weights_array = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)
weights_dict = {i: weights_array[i] for i in range(len(np.unique(y)))}


# ---- MOYENNE des réplicats
#---------------------------------
souches = pd.unique(label_df['strain'] )#nb de souches uniques

# Création de la matrice avec la moyenne pour chaque souche
data_mean=np.zeros((len(souches),data.shape[1])) #nb de colonnes, exclude species and strain
feature_columns = [col for col in data.columns if col.startswith('Feature')]

for x in range(len(souches)):
    data_mean[x,:]=np.mean(data.iloc[x*9:(x+1)*9, :][feature_columns], axis = 0)  #moyenne par colonne



#creation d'une liste avec les indices des réplicats numéro 1:
z=0
index_R1= [None]* len(souches)
for i in range(len(souches)):
    #print(z)
    index_R1[i] = z
    z=z+9

print(index_R1,data_mean)

# split data_mean
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(data_mean, y[index_R1], test_size = 0.25, random_state = 1, stratify = y[index_R1])


#X_train_m=data_mean
#y_train_m=y[index_R1]

#data_test= np.genfromtxt('/Users/anath/Master_SSD/Semestre3/Fouille_donnees/dataset_challenge/spectra-train.csv', delimiter = ";")
#label_test_df= pd.read_csv('/Users/anath/Master_SSD/Semestre3/Fouille_donnees/dataset_challenge/meta-train.csv', delimiter = ";")

#---- MOYENNE des réplicats sur meta-test et standar
#---------------------------------
#souches_test = pd.unique(label_test_df['strain'] )#nb de souches uniques
# Création de la matrice avec la moyenne
#souches_test.shape
#data_test_mean=np.zeros((len(souches_test),data_test.shape[1]))

#for x in range(len(souches_test)):
   # data_test_mean[x,:]=np.mean(data_mean[x*9:(x+1)*9, :], axis = 0)

#X_test_m=data_mean

[0, 9, 18, 27, 36, 45, 54, 63, 72, 81, 90, 99, 108, 117, 126, 135, 144, 153, 162, 171] [[ 3.55420969e+00  9.88477963e+00  6.71569752e+00  6.12668285e+00
   1.24052918e+00  1.38093599e+00  6.35813771e-02  8.52975536e+00
   6.25118872e+00  6.87585392e+00 -2.29608915e-01  9.95357736e+00
   8.56109497e+00  2.47109987e+00  1.51219479e+00  1.62778905e+00
   3.57344357e+00  5.03506021e+00  4.40754349e+00  3.40919759e+00
   6.03512208e+00  1.74972630e+00  2.51122890e+00  3.97253342e+00
   4.91973455e+00  7.87754646e+00  2.51299210e+00  5.21347726e+00
   6.08017915e+00  2.59419598e-01  6.73389822e+00  1.37816196e+00
   8.73633510e-01  9.96569594e+00  9.68163933e+00  8.58715735e+00
   3.15890629e+00  1.07478488e+00  7.53482824e+00  4.36866843e+00
   7.84308795e-01  4.50240106e+00  1.63676743e-02  8.90058935e+00
   2.95308762e+00  6.52513246e+00  2.99786190e+00  5.17865440e+00
   5.51191204e+00  1.59107273e+00]
 [ 7.63960465e+00  4.77416973e+00  8.66087892e-01  5.64728842e+00
   6.03132437e+00  7

In [34]:
svm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(class_weight='balanced'))

])

# define grid of parameter
C_grid = np.logspace(-2,2, 10)
gamma_grid = [0.1,0.5, 1, 2]
param_grid = [
    {'svm__decision_function_shape': ['ovr','ovo'],'svm__kernel' : ['linear'], 'svm__C' : C_grid},
    {'svm__decision_function_shape': ['ovr','ovo'],'svm__kernel' : ['rbf'], 'svm__C' : C_grid, 'svm__gamma' : gamma_grid}
]
# optimise parameter
grid_search = GridSearchCV(svm_pipe, param_grid, cv= 3)
grid_search.fit(X_train_m,y_train_m)
# show results
print("The best parameters are %s with a score of %0.2f"
     % (grid_search.best_params_, grid_search.best_score_))

The best parameters are {'svm__C': np.float64(1.6681005372000592), 'svm__decision_function_shape': 'ovr', 'svm__gamma': 0.1, 'svm__kernel': 'rbf'} with a score of 0.20


In [35]:
pred=grid_search.best_estimator_.predict(X_test_m)


In [37]:
pred

array([2, 3, 3, 1, 0])

In [38]:
y_test_m

array([2, 0, 4, 3, 1])